In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
books_url = 'https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/books.csv'
ratings_url = 'https://raw.githubusercontent.com/zygmuntz/goodbooks-10k/master/ratings.csv'

books_df = pd.read_csv(books_url)
ratings_df = pd.read_csv(ratings_url)

In [3]:
books_df.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,"The Hunger Games (The Hunger Games, #1)",eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,Harry Potter and the Sorcerer's Stone (Harry P...,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,"Twilight (Twilight, #1)",en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [4]:
books_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   book_id                    10000 non-null  int64  
 1   goodreads_book_id          10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [5]:
ratings_df.head()

,user_id,book_id,rating
0,1,258,5
1,2,4081,4
2,2,260,5
3,2,9296,5
4,2,2318,3


In [6]:
ratings_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5976479 entries, 0 to 5976478
Data columns (total 3 columns):
 #   Column   Dtype
---  ------   -----
 0   user_id  int64
 1   book_id  int64
 2   rating   int64
dtypes: int64(3)
memory usage: 136.8 MB


In [7]:
ratings_df.sample(5)

,user_id,book_id,rating
1241594,13798,31,5
2777510,35364,853,3
2338795,28462,985,5
5111488,52080,326,4
559799,1724,526,4


Create a list of your book ratings from the existing books in the dataset

In [8]:
my_list = {'Martin Eden': 5,
            'Pet Sematary': 5,
            'One Hundred Years of Solitude': 5,
            'Ham on Rye': 5,
            'The Grapes of Wrath': 4, 
            "Cat's Cradle": 5,
            'Crime and Punishment': 4,
            'The Trial': 4}

In [9]:
# Create dataframe for new user (me)
user_books = pd.DataFrame(columns=['title', 'rating'], data=my_list.items())
user_books

,title,rating
0,Martin Eden,5
1,Pet Sematary,5
2,One Hundred Years of Solitude,5
3,Ham on Rye,5
4,The Grapes of Wrath,4
5,Cat's Cradle,5
6,Crime and Punishment,4
7,The Trial,4


In [10]:
# Add book_id from books_df
new_user = pd.merge(user_books, books_df, on='title', how='inner')
new_user = new_user[['book_id', 'title', 'rating']].sort_values(by='book_id')
new_user

,book_id,title,rating
2,94,One Hundred Years of Solitude,5
4,131,The Grapes of Wrath,4
6,177,Crime and Punishment,4
5,297,Cat's Cradle,5
1,305,Pet Sematary,5
7,614,The Trial,4
3,1444,Ham on Rye,5
0,7068,Martin Eden,5


In [11]:
other_users = ratings_df[ratings_df['book_id'].isin(new_user['book_id'].values)]
other_users


,user_id,book_id,rating
38,4,297,4
49,8,614,4
160,11,94,5
182,8,177,5
276,4,131,4
...,...,...,...
5973753,39833,94,1
5973913,48436,297,4
5974295,51542,94,4
5974628,7607,94,5


In [12]:
other_users['user_id'].nunique()


18637

In [13]:
# Sort users by count of most mutual books with me
users_mutual_books = other_users.groupby(['user_id'])
users_mutual_books = sorted(users_mutual_books, key=lambda x: len(x[1]), reverse=True)
users_mutual_books[0]

(47897,          user_id  book_id  rating
 4378842    47897      177       5
 4961030    47897      131       5
 4961036    47897     1444       5
 4961076    47897       94       5
 4961530    47897      297       5
 4961537    47897     7068       5
 4961543    47897      614       4)

In [14]:
top_users = users_mutual_books[:100]

In [15]:
# Pearson correlation
from scipy.stats import pearsonr

pearson_corr = {}

for user_id, books in top_users:
    # Books should be sorted
    books = books.sort_values(by='book_id')
    book_list = books['book_id'].values

    new_user_ratings = new_user[new_user['book_id'].isin(book_list)]['rating'].values 
    user_ratings = books[books['book_id'].isin(book_list)]['rating'].values

    corr = pearsonr(new_user_ratings, user_ratings)
    pearson_corr[user_id] = corr[0]


In [16]:
# Get top50 users with the highest similarity indices
pearson_df = pd.DataFrame(columns=['user_id', 'similarity_index'], data=pearson_corr.items())
pearson_df = pearson_df.sort_values(by='similarity_index', ascending=False)[:50]
pearson_df

,user_id,similarity_index
33,11862,1.000000
31,11274,0.912871
57,27747,0.912871
32,11356,0.872872
20,5426,0.866025
43,17944,0.763763
9,1027,0.763763
50,21646,0.745356
26,8616,0.720577
1,10898,0.707107


In [17]:
# Get all books for these users and add weighted book's ratings
users_rating = pearson_df.merge(ratings_df, on='user_id', how='inner')
users_rating['weighted_rating'] = users_rating['rating'] * users_rating['similarity_index']
users_rating

,user_id,similarity_index,book_id,rating,weighted_rating
0,11862,1.000000,4,4,4.000000
1,11862,1.000000,8,4,4.000000
2,11862,1.000000,5,3,3.000000
3,11862,1.000000,14,3,3.000000
4,11862,1.000000,50,3,3.000000
...,...,...,...,...,...
6752,32921,0.080064,223,3,0.240192
6753,32921,0.080064,3418,1,0.080064
6754,32921,0.080064,184,4,0.320256
6755,32921,0.080064,7879,3,0.240192


In [18]:
# Calculate sum of similarity index and weighted rating for each book
grouped_ratings = users_rating.groupby('book_id').sum()[['similarity_index', 'weighted_rating']]

In [19]:
recommend_books = pd.DataFrame()

# Add average recommendation score
recommend_books['avg_reccomend_score'] = grouped_ratings['weighted_rating']/grouped_ratings['similarity_index']
recommend_books['book_id'] = grouped_ratings.index
recommend_books = recommend_books.reset_index(drop=True)

# Left books with the highest score
recommend_books = recommend_books[(recommend_books['avg_reccomend_score'] == 5)]
recommend_books

,avg_reccomend_score,book_id
129,5.0,145
130,5.0,147
133,5.0,151
157,5.0,185
196,5.0,245
...,...,...
1887,5.0,9463
1891,5.0,9561
1892,5.0,9605
1898,5.0,9722


In [20]:
# Let's see our recomendations
recommendation = books_df[books_df['book_id'].isin(recommend_books['book_id'])][['authors', 'title', 'book_id']].sample(10)
recommendation

,authors,title,book_id
3103,David Herbert Donald,Lincoln,3104
6862,"Flannery O'Connor, Robert Fitzgerald",Everything That Rises Must Converge: Stories,6863
1544,"A.A. Milne, Ernest H. Shepard","The House at Pooh Corner (Winnie-the-Pooh, #2)",1545
4119,Richard Dawkins,The Blind Watchmaker: Why the Evidence of Evol...,4120
8078,"Italo Calvino, William Weaver, Ανταίος Χρυσοστ...",Cosmicomics,8079
8665,Paul Theroux,Dark Star Safari: Overland from Cairo to Cape ...,8666
347,E. Lockhart,We Were Liars,348
2208,William Shakespeare,The Complete Works,2209
6692,Robert Graysmith,Zodiac,6693
1592,Raymond E. Feist,"Magician: Apprentice (The Riftwar Saga, #1)",1593
